In [20]:
import abc
import ollama

class Agent(abc.ABC):
    def __init__(self):
        self.sys_message = {'role': 'system', 'content': self.system_prompt()}
        self.user_message_default = "Introduce yourself."

    @abc.abstractmethod
    def system_prompt(self):
        """Return the system prompt content"""
        pass
    
    @abc.abstractmethod
    def user_prompt(self):
        pass

    def chat(self, user_input=None, model="llama3"):

        if user_input:
            user_content = self.user_prompt(user_input)
        else:
            user_content = self.user_message_default

        print(user_content)

        user_message = {'role': 'user', 'content': user_content}

        response = ollama.chat(model=model, messages=[self.sys_message, user_message])
        return response['message']['content'].strip()


In [53]:
class Preprocessor(Agent):
    def system_prompt(self):
        return "You are an expert editor. You preprocess material for translation with no additional output."
    
    def user_prompt(self, user_input):
        return f"""Preprocess the following: {user_input}
                    Corrected input: 
                """

In [44]:
class Translator(Agent):
    def system_prompt(self):
        return "You are an expert translator. Translate clearly and concisely with no additional output."
    
    def user_prompt(self, user_input):
        return f"Translate the following to German, with no additional output:{user_input}"


In [23]:
class Evaluator(Agent):
    def system_prompt(self):
        return "You are an expert in translation evaluation."
    
    def user_prompt(self, user_input):
        return f"""Score the following translation on a continuous scale from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar" but provide no additional output.

                Source: {user_input['source']}
                Target: {user_input['target']}
                """

In [41]:
class PostEditor(Agent):
    def system_prompt(self):
        return """"You are an expert translation editor. You provide improved translations based on an original translation and a score from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar". You only provide an improved translation."""
    
    def user_prompt(self, user_input):
        return f"""Improv the following translation:
        Source: {user_input['source']},
        Translation: {user_input['translation']}
        Score: {user_input['score']}

        Only output the corrected target translation.
        """

In [54]:
prepro = Preprocessor()
translator = Translator()
eval = Evaluator()
post = PostEditor()

In [55]:
prepro.chat(user_input='Im eat cake.')

Preprocess the following: Im eat cake.
                    Corrected input: 
                


'Corrected input:\n\nI eat cake.'

In [47]:
translator.chat(user_input='I am eating cake.')

Translate the following to German, with no additional output:I am eating cake.


'Ich esse Kuchen.'

In [25]:
eval.chat({'source': 'I am a dog', 'target': 'Ich bin ein Berliner'})

Score the following translation on a continuous scale from 0 to 100, where score of zero means
                "no meaning preserved" and score of one hundred means "perfect meaning and grammar" but provide no additional output.

                Source: I am a dog
                Target: Ich bin ein Berliner
                


'30.5'

In [43]:
post.chat({'source': 'I am a dog', 'translation': 'Ich bin ein Berliner', 'score':30.5})

Improv the following translation:
        Source: I am a dog,
        Translation: Ich bin ein Berliner
        Score: 30.5

        Only output the corrected target translation.
        


'Ich heiße ein Hund.'